<a href="https://colab.research.google.com/github/MaxPr1me/Ontario_Scope2_EmissionFactor/blob/main/Emission_Factors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clear any previous variables or outputs (not needed in Python, but done here for clarity)
# This is just a visual reset equivalent to `clc; clear all; close all;` in MATLAB
%reset -f

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

In [ ]:
# Mount Google Drive to access the IESO data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set the base path relative to where the notebook is stored
# The notebook is in 'My Drive/Code - Emission Factors', so we append the relative path to the data
base_path = '/content/drive/My Drive/Code - Emission Factors/IESO Data/'

# Define file path templates for the different data types
supply_template = 'Supply/GOC-{}.xlsx'
demand_template = 'Demand/PUB_DemandZonal_{}.csv'
generator_list_template = 'Generator_List.xlsx'
transmission_template = 'Transmission/PUB_IntertieScheduleFlowYear_{}.csv'

# Initialize an empty dictionary to store data for each year
data = {}

In [ ]:
# Loop through each year from 2016 to 2023
for year in range(2016, 2024):
    # Generate file paths based on the year using the relative base path
    supply_file = os.path.join(base_path, supply_template.format(year))
    demand_file = os.path.join(base_path, demand_template.format(year))
    generator_list_file = os.path.join(base_path, generator_list_template)
    transmission_file = os.path.join(base_path, transmission_template.format(year))

    # Load files and handle missing data gracefully
    try:
        # Check if the files exist before trying to load them
        if os.path.exists(supply_file) and os.path.exists(demand_file) and os.path.exists(generator_list_file) and os.path.exists(transmission_file):
            # Load supply data (from Excel)
            supply_data = pd.read_excel(supply_file)
            # Load demand data (from CSV)
            demand_data = pd.read_csv(demand_file)
            # Load generator list
            generator_list = pd.read_excel(generator_list_file)
            # Load transmission data
            transmission_data = pd.read_csv(transmission_file)

            # Store the data in a dictionary
            data[year] = {
                'supply': supply_data,
                'demand': demand_data,
                'generator_list': generator_list,
                'transmission': transmission_data
            }
        else:
            print(f"Files for {year} are missing. Skipping this year.")

    except Exception as e:
        print(f"Error loading data for {year}: {e}")

Files for 2016 are missing. Skipping this year.
Files for 2017 are missing. Skipping this year.


In [ ]:
# Once loaded, let's clean and process the data

# Function to replace NaN values with 0 in all numerical columns
def clean_data(df):
    return df.fillna(0)

for year in data:
    # Clean supply, demand, and transmission data
    data[year]['supply'] = clean_data(data[year]['supply'])
    data[year]['demand'] = clean_data(data[year]['demand'])
    data[year]['transmission'] = clean_data(data[year]['transmission'])

In [ ]:
# Now let's extract the timestamps from the 'txt' variable equivalent in the supply data.
# Assuming that the first column of the supply data contains the date-time information

for year in data:
    try:
        # Convert the 'Date' column to a datetime object
        time_data = pd.to_datetime(data[year]['supply'].iloc[:, 0], errors='coerce')  # Coerce errors to handle bad dates

        # Add the hours from the 'Hour' column as a time delta
        hours_data = pd.to_numeric(data[year]['supply'].iloc[:, 1], errors='coerce')
        if hours_data.notnull().all():
            time_data = time_data + pd.to_timedelta(hours_data - 1, unit='h')  # Subtract 1 to account for 1-based hour index

        # Store the processed time data
        data[year]['time'] = time_data

    except Exception as e:
        print(f"Error processing time data for {year}: {e}")

In [ ]:
# Define regions and technologies
regions = ['Northwest', 'Northeast', 'Ottawa', 'East', 'Toronto', 'Essa', 'Bruce', 'Southwest', 'Niagara', 'West']
technologies = ['Biofuel', 'Hydro', 'Natural Gas', 'Nuclear', 'Solar', 'Wind']

# Extract demand data for each zone (from demand CSV file)
demand_data = data[year]['demand']

# Create a dictionary to hold the demand data for each region
demand_by_region = {}
for i in range(1, 11):  # 10 regions
    demand_by_region[i] = demand_data.iloc[:, 2 + i].to_numpy()  # Columns 3 to 12 for each region's demand

In [ ]:
# Initialize generator data structure for each region and technology
# Gen will be a dictionary of dictionaries, where Gen[region][technology] stores the data
gen = {region: {tech: np.zeros(len(data[year]['time'])) for tech in technologies} for region in regions}

# Generator count
gencount = 0

# Loop through generator data and map it to the correct region and technology
for col_idx in range(3, len(data[year]['supply'].columns)):  # Starting at column 4 (index 3)
    generator_name = data[year]['supply'].columns[col_idx]

    # Loop through the generator list (assuming it's stored in data[year]['generator_list'])
    for gen_idx, row in data[year]['generator_list'].iterrows():
        if generator_name == row[0]:  # Compare generator names (column 0)
            # Determine the region
            region_name = row[2]  # Assuming region is in column 3
            if region_name in regions:
                region = region_name

            # Determine the technology
            tech_name = row[1]  # Assuming technology is in column 2
            if tech_name in technologies:
                technology = tech_name

            # Add the generator data to the correct region and technology
            gen[region][technology] = np.column_stack((gen[region][technology], data[year]['supply'].iloc[:, col_idx]))

            # Replace NaN values with 0
            gen[region][technology][np.isnan(gen[region][technology])] = 0

            # Increment the generator count
            gencount += 1
            break  # Move to the next generator

<ipython-input-13-8c019e74cd4f>:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if generator_name == row[0]:  # Compare generator names (column 0)
<ipython-input-13-8c019e74cd4f>:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  region_name = row[2]  # Assuming region is in column 3
<ipython-input-13-8c019e74cd4f>:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tech_name = row[1]  # Assuming technology is in column 2
